In [1]:
!pip install pygame
!pip install neat-python
import pygame
import neat
import os
import time
import random
pygame.font.init()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
screen_width = 550
screen_height = 800
white = (255,255,255)
GEN = 0

bird_imgs = [pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird" + str(x) + ".png")))
for x  in range(1,4)]
pipe_img = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "pipe.png")))
base_img = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "base.png")))
bg_img =   pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "bg.png")))
Stat_font = pygame.font.SysFont('comicsans',50)


class Bird:
    imgs = bird_imgs
    max_rotation = 25 #tilting of bird up or down
    rot_velocity = 20 #how much rotation in 1 frame
    animation_time = 5 #how long animation is shown(used for flapping effect)
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.tilt = 0
        self.tick_count = 0
        self.vel = 0
        self.height = self.y
        self.img_count = 0
        self.img = self.imgs[0]
    
    def jump(self):
        self.vel = -10.5 # Go upward in X-direction
        self.tick_count = 0
        self.height = self.y
    
    def move(self):
        self.tick_count += 1
        # for downward acceleration.Uses the physics equation s = u*t + 1/2 * a * t ** 2
        d = self.vel*self.tick_count + 1.5*self.tick_count**2
#               terminal velocity
#           For those who didn't understand the formula for displacement, here are the displacement values using that formula with respect to the frame going on (that is with respect to the self.tick_count) :-
#             Current Frame: 1
#             Current Velocity -9.0
#             Current Frame: 2
#             Current Velocity -15.0
#             Current Frame: 3
#             Current Velocity -18.0
#             Current Frame: 4
#             Current Velocity -18.0
#             Current Frame: 5
#             Current Velocity -15.0
#             Current Frame: 6
#             Current Velocity -9.0
#             Current Frame: 7
#             Current Velocity 0.0
#             Current Frame: 8
#             Current Velocity 12.0
#             Current Frame: 9
#             Current Velocity 27.0


        if d >= 16:
            d= 16
        if d < 0:
            d -= 2
        
        self.y = self.y + d
        
        if d < 0 or self.y < self.height + 50: #condition to check if the bird is below its original pos (to change animation)
            if self.tilt < self.max_rotation:
                self.tilt = self.max_rotation #tilting upward animation 
        else: #when bird is going down
            if self.tilt > -90:          
                self.tilt -= self.rot_velocity #animation like nose diving
            
    def draw(self, win):
        self.img_count += 1
        
        if self.img_count < self.animation_time:
            self.img = self.imgs[0]
        elif self.img_count < self.animation_time * 2:
            self.img = self.imgs[1]
        elif self.img_count < self.animation_time * 3:
            self.img = self.imgs[2]    
        elif self.img_count < self.animation_time * 4:
            self.img = self.imgs[1]
        elif self.img_count == self.animation_time * 4 +1:
            self.img = self.imgs[0]
            self.img_count = 0
        
        if self.tilt <= -80:
            self.img = self.imgs[1]
            self.img_count = self.animation_time * 2
        
        # now the bird goes up and down. Let's add some tilting.
        
        rotated_image = pygame.transform.rotate(self.img, self.tilt)
        new_rect = rotated_image.get_rect(center = self.img.get_rect(topleft = (self.x, self.y)).center)
        win.blit(rotated_image, new_rect.topleft)
    
    def get_mask(self):
        return pygame.mask.from_surface(self.img)

    
class Pipe:
    GAP = 200
    vel = 5
    
    def __init__(self,x):
        self.x = x
        self.height = 0
        self.top = 0
        self.bottom = 0
        self.PIPE_TOP = pygame.transform.flip(pipe_img, False, True)
        self.PIPE_BOTTOM = pipe_img
        
        self.passed = False
        self.set_height()
    
    def set_height(self):
        self.height = random.randrange(50,450)
        self.top = self.height - self.PIPE_TOP.get_height() #to set the bottom of top pip(head of pipe) to be in position
        self.bottom = self.height + self.GAP
    
    def move(self):
        self.x -= self.vel
        
        
    def draw(self,win):
        win.blit(self.PIPE_TOP,(self.x, self.top))
        win.blit(self.PIPE_BOTTOM,(self.x, self.bottom))
        
    def collide(self, bird):
        bird_mask = bird.get_mask() 
        top_mask = pygame.mask.from_surface(self.PIPE_TOP)
        bottom_mask = pygame.mask.from_surface(self.PIPE_BOTTOM)
        
        top_offset = (self.x - bird.x, self.top - round(bird.y)) # checking if bird and pipe are not colliding
        bottom_offset = (self.x - bird.x, self.bottom - round(bird.y))
        
        # Now if they collide, let's find the colliding point
        bottom_point = bird_mask.overlap(bottom_mask, bottom_offset)
        top_point = bird_mask.overlap(top_mask, top_offset) # both return some value if collided
        
        if bottom_point or top_point: #if we get any values?
            return True
        
        return False

class Base:
    vel = 5
    width = base_img.get_width()
    img = base_img
    
    def __init__(self, y):
        self.y = y
        self.x1 = 0
        self.x2 = self.width
    
    def move(self): #since we do not have an infinitely long image, we just repeat the image
        self.x1 -= self.vel
        self.x2 -= self.vel
        
        if self.x1 + self.width < 0:
            self.x1 = self.x2 + self.width
        if self.x2 + self.width < 0:
            self.x2 = self.x1 + self.width
        
    def draw(self, win):
        win.blit(self.img, (self.x1, self.y))
        win.blit(self.img, (self.x2, self.y)) 

def draw_window(win, birds, pipes, base, score, gen):
    win.blit(bg_img, (0,0))
    
    for pipe in pipes:
        pipe.draw(win)
    
    text = Stat_font.render("Score : " + str(score), 1, white)
    win.blit(text, (0, 10))
    
    text = Stat_font.render("Generation : " + str(gen-1), 1, white)
    win.blit(text, (0, 45))
    
    text = Stat_font.render("Alive: " + str(len(birds)), 1, white)
    win.blit(text, (0, 80))
    
    base.draw(win)
    for bird in birds:
        bird.draw(win)
    pygame.display.update()

    
def main(genomes, config):
    win = pygame.display.set_mode((screen_width, screen_height))
    birds = []
    nets = []
    ge = []
    global GEN
    GEN += 1
    
    for _, g in genomes: # genomes have a tuple valy of ID and genenome object. we are only intrested in the object
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        birds.append(Bird(230, 350))
        g.fitness = 0
        ge.append(g)
    
    base = Base(730)
    pipes = [Pipe(600)]
    run = True
    clock = pygame.time.Clock()
    score = 0
    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False  
                pygame.quit()

     
        pipe_ind  = 0
        if len(birds) > 0:
            if len(pipes) > 1 and birds[0].x > pipes[0].x + pipes[0].PIPE_TOP.get_width():
                pipe_ind = 1
        
        else:
           # run = False
            break
                
                
        for x, bird in enumerate(birds):
            bird.move()
            ge[x].fitness += 0.1
            output = nets[x].activate((bird.y, abs(bird.y - pipes[pipe_ind].height), abs(bird.y - pipes[pipe_ind].bottom))) #distance between bird position and the top and bottom of pipes
            
            if output[0] > 0.5:
                bird.jump()
                
        add_pipe = False
        rem = []
        for pipe in pipes:
            for x, bird in enumerate(birds):
                if pipe.collide(bird):
                    ge[x].fitness -= 1
                    birds.pop(x)
                    nets.pop(x)
                    ge.pop(x)
                
                if not pipe.passed and pipe.x < bird.x:
                    pipe.passed = True
                    add_pipe = True
                
                    
            if pipe.x + pipe.PIPE_TOP.get_width() < 0:
                rem.append(pipe)

            pipe.move()
       
        if add_pipe:
            score += 1
            for g in ge:
                g.fitness += 5
            pipes.append((Pipe(600)))
        
        for r in rem:
            pipes.remove(r)
        
        for x, bird in enumerate(birds):
            if bird.y + bird.img.get_height() >= 730 or bird.y < 0:
                birds.pop(x)
                nets.pop(x)
                ge.pop(x)
                
        
        base.move()
        draw_window(win, birds, pipes, base, score, GEN)



def run(config_path):
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, 
                               neat.DefaultSpeciesSet, neat.DefaultStagnation,
                                config_path)
    p = neat.Population(config)
    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    
    winner = p.run(main,50) #Best genome 
    
    print('\nBest genome:\n{!s}'.format(winner))
    
if __name__ == "__main__":
    local_dir = os.path.dirname(os.path.abspath("__file__"))
    config_path = os.path.join(local_dir, "config-feedforward.txt")
    run(config_path)



 ****** Running generation 0 ****** 

Population's average fitness: 6.97500 stdev: 11.87118
Best fitness: 56.50000 - size: (1, 3) - species 1 - id 17
Average adjusted fitness: 0.086
Mean genetic distance 1.175, standard deviation 0.389
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    20     56.5    0.086     0
Total extinctions: 0
Generation time: 12.517 sec

 ****** Running generation 1 ****** 

Population's average fitness: 4.69000 stdev: 3.45744
Best fitness: 18.50000 - size: (1, 3) - species 1 - id 17
Average adjusted fitness: 0.148
Mean genetic distance 1.809, standard deviation 0.571
Population of 20 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    16     18.5    0.148     1
     2    0     4       --       --     0
Total extinctions: 0
Generation time: 4.916 sec (8.716 average)

 ****** Running generation 2 ****** 



error: display Surface quit